# Padding Oracle 
- When a decrypted CBC ciphertext ends in an invalid pad the web server returns a 403 error code (forbidden request). When the CBC padding is valid, but the message is malformed, the web server returns a 404 error code (URL not found).
```
http://crypto-class.appspot.com/po?er="your ciphertext here"
```
- The first ciphertext block is random IV,  the decrypted text block is ascii encoded
- the ciphertext following the `"po?er="` is a hex encoded AES CBC encryption with a random IV of some secret data about Alice's session.

In [ ]:
import urllib3 as ul

In [ ]:
BLOCKSIZE = 16

AZ = [i for i in range(ord('A'), ord('Z') + 1)] 
space = [ord(' ')]
az = [i for i in range(ord('a'),ord('z') +1)]
paddings = [i for i in range(1, 17)]

misc1 = [i for i in range(17, 32)] + [i for i in range(33, 65)]
misc2 = [i for i in range(91, 97)] + [i for i in range(123, 128)]

ALL = paddings + space + az  + AZ + misc1 + misc2


def xor(x, y, z):
    assert len(x) == len(y) == len(z)
    a = int.from_bytes(x, "big")
    b = int.from_bytes(y, "big")
    c = int.from_bytes(z, "big")
    r = a ^ b ^ c
    return r.to_bytes(len(x), "big")


# Target: "http:domain.com/po?er="
class PaddingOracle:
    def __init__(self, target):
        self.target = target
        self.http = ul.PoolManager()
    
    # ct: string representing hex encoded 
    # 4 * 16 * 2 == 128 characters in length
    # 4 blocks of ciphertxt, 1 block IV, 3 blocks ciphertext
    def decrypt4blocks(self, ct, debug=True):
        
        assert len(ct) == 128
        assert self.status_query(ct) == 200

        iv, c0, c1, c2 = ct[:32], ct[32:64], ct[64:96], ct[96:]
        
        print("Decrypting...")
        m0 = self.decrypt_block(c0, iv)
        print(" > ", m0)
        
        m1 = self.decrypt_block(c1, c0)
        print(" > ",  m1)
        
        m2 = self.decrypt_block(c2, c1)
        print(" > ", m2)    
        return m0 + m1 + m2
    
    def decrypt_block(self, c, c0_hex):
        m = bytearray(BLOCKSIZE)
        c0 = bytes.fromhex(c0_hex)
        
        for i in range(1, BLOCKSIZE + 1):
            self.overwrite_and_send_byte(m, c, i, c0)
        return m
    
    # Overwrites one byte in message m for each iteration
    def overwrite_and_send_byte(self, m, c, i, c0):
        
        n = bytes([i for _ in range(BLOCKSIZE)])
        CURRENT = BLOCKSIZE - i
        
        for g in ALL:
            
            m[CURRENT] = g 
            q = xor(n, m, c0).hex() + c
                            
            if self.is_valid(q) is True:
                print(chr(g), end="_")
                return
            
        raise ValueError("Unable to find byte")
            
    def is_valid(self, q):
        r = self.http.request('GET', self.target + q, retries=False)
        return r.status != 403 
    
    def status_query(self, q):
        return self.http.request('GET', self.target + q, retries=False).status

In [ ]:
TARGET = 'http://crypto-class.appspot.com/po?er='
CIPHERTEXT = "f20bdba6ff29eed7b046d1df9fb7000058b1ffb4210a580f748b4ac714c001bd4a61044426fb515dad3f21f18aa577c0bdf302936266926ff37dbf7035d5eeb4"

In [ ]:
po = PaddingOracle(TARGET)
message = po.decrypt4blocks(CIPHERTEXT)

In [ ]:
print(message)

In [ ]:
ct1 = "4ca00ff4c898d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81"
ct2 = "5b68629feb8606f9a6667670b75b38a5b4832d0f26e1ab7da33249de7d4afc48e713ac646ace36e872ad5fb8a512428a6e21364b0c374df45503473c5242a253"
pt1 = "Basic CBC mode encryption needs padding."
pt2 = "Our implementation uses rand. IV"

TARGET = "http://localhost:9000/po?er="

po = PaddingOracle(TARGET)
message1 = po.decrypt4blocks(ct1)
print(message1)

message2 = po.decrypt4blocks(ct2)
print(message2)